In [15]:

import os
import re
import urllib.request
from flask import *
import sqlite3
from werkzeug.utils import secure_filename
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from newspaper import Article
import nltk
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity


vx=[]
out={}
v=[]
title=[]
nltk.download('stopwords')
def read_article(file):
    
    article = file.split(". ")
    sentences = []

    for sentence in article:
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))
    sentences.pop() 
    
    return sentences

def sentence_similarity(sent1, sent2, stopwords=None):
    if stopwords is None:
        stopwords = []
 
    sent1 = [w.lower() for w in sent1]
    sent2 = [w.lower() for w in sent2]
 
    all_words = list(set(sent1 + sent2))
 
    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)
    
    # build the vector for the first sentence
    for w in sent1:
        if w in stopwords:
            continue
        vector1[all_words.index(w)] += 1
 
    # build the vector for the second sentence
    for w in sent2:
        if w in stopwords:
            continue
        vector2[all_words.index(w)] += 1
 
    return 1 - cosine_distance(vector1, vector2)
 
def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
 
    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2: #ignore if both are same sentences
                continue 
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix


def generate_summary(file_name, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text anc split it
    sentences =  read_article(file_name)

    # Step 2 - Generate Similary Martix across sentences
    sentence_similarity_martix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity martix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_martix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)    
    #print("Indexes of top ranked_sentence order are ", ranked_sentence)    

    for i in range(top_n):
        try:
          summarize_text.append(" ".join(ranked_sentence[i][1]))
        except:
            pass

    # Step 5 - Offcourse, output the summarize texr
    return ". ".join(summarize_text)
    
def simlaritytitle(simple):
    try:
        if simple not in vx:
            model = SentenceTransformer('nli-distilroberta-base-v2')
            sentences=title
            sentence_embeddings = model.encode(sentences)
            similarity_score = cosine_similarity(
            [sentence_embeddings[simple]],
            sentence_embeddings[0:]
        ).flatten()
            if simple not in vx:
                out[simple]=[]
            for k in range(len(sentences[0:])):
                if(similarity_score[k]>0.8) and k!=simple:
                    if k not in vx:
                        vx.append(k)
                        out[simple].append(k)
    except Exception as e :
        print(e)
        pass    

def data(site,count):
    op = urlopen(site)  # Open that site
    rd = op.read()  # read data from site
    op.close()  # close the object
    sp_page = soup(rd, 'xml')  # scrapping data from site
    news_title = sp_page.find_all("title")  # finding news
    news_link = sp_page.find_all("link")  # finding news
    news_pubDate = sp_page.find_all("pubDate")  # finding news
    news_description = sp_page.find_all("description")  # finding news
    news_source = sp_page.find_all("source")  # finding news
    r=1
    for k in range(1,count+1):
        ne={}
        title.append(news_title[k].get_text())
        ne["title"]=(news_title[k].get_text())
        ne["link"]=[news_link[k].get_text()]
        ne["pubDate"]=news_pubDate[k].get_text()
        ne["description"]=news_description[k].get_text()
        ne["source"]=news_source[k].get_text()
        v.append(ne)
        r+=1
    for kx in range(len(v)):
        print(v[kx])
        simlaritytitle(kx)
    vr=[]
    for k in out.keys():
        print((len(out[k])!=0) and  k not in vx)
        if(len(out[k])!=0) and  k not in vx:
            for x in out[k]:
                v[k]["link"].append(v[x]["link"])
            vr.append(v[k])
        else:
            vr.append(v[k])
        
    print(vr)
    si=[]
    for k in vr:
        print(k)
        va=""
        for link in k["link"]:
            try:
                wor=[]
                wor.append(k["title"])
                wor.append(k["link"])
                wor.append(k["pubDate"])
                wor.append(k["source"])
                r = requests.get(link, timeout=5)
                site=(r.url)
                op = urlopen(site)  # Open that site
                sp_page = soup(op, 'html.parser')  # scrapping data from site
                va+="".join(sp_page.get_text().split("\n\n\n\n"))
                va+="................................................................."
                op.close()
                va=va.replace("\n\n"," ")
                b=va.split("\n")
                c="**".join(b)
                r=c.split("**")
                vae=""
                for x in r:
                    if(len(x)>200):
                        vae+=x+"\n"
                # wor.append('Orginal content:'+vae)
                wor.append("Total Words:"+str(len(vae.split(" "))))
                
                var=generate_summary(vae,2)
                wor.append('Summarised content:'+var)
                wor.append("Total Words:"+str(len(var.split(" "))))
                si.append(wor)
            except Exception as e:
                pass
        return si
def dataser(value):
    v=value["news"]
    count=value["count"]
    print(v,count)
    if v=="headlines":
        url="https://news.google.com/news/rss"
    else:
        url = 'https://news.google.com/rss/search?q={}'.format(v)
    
    import json
    x=data(url,int(count))  
    y=open("out.txt","w",encoding="utf-8")
    y.write(json.dumps(x))
    y.close()
    x=open("out.txt","r").read()
    print("completed")
    return x
value={"news":"headlines","count":5}
dataser(value)
import json
x=open("out.txt","r").read()
xr=json.loads(x)
for n in xr:
    print(n)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rohin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


headlines 5
{'title': 'Google News', 'link': ['https://news.google.com/'], 'pubDate': 'Mon, 21 Oct 2024 04:47:00 GMT', 'description': '<ol><li><a href="https://news.google.com/rss/articles/CBMi2wFBVV95cUxPb3QwR1F3RUExX0l6ZWstZFB4SG9vbXVkQXFFckpwdHYyQlhjUzhOaEVkNU14QjB0bHE2ZDNzV3BQZEZXdWdCeDR0OU5LTlNRRUg1OGpJZGhDcG1ZNnNKOVVfZnBWZk9qQkVsV18yWUktcEc0dXE4QVpfZE9RMzFKVEFrQ0kxUUFQYVl6djVYckl3T3ZNVDZvcVhZR2g5MzlBQkNtbW52Vmt4RGlleHZsVkROanM0a2tzTmdrUXB5MkVUOXFrY245anJfV2xuS1Z2SDRjYmc0S2pvQm_SAeABQVVfeXFMTl85MlNiZ2F5SEg0NFJ5eUpEVFpHOEI1WUhyMDlHUjdfcURLYVFmS0pXeTNHYnlLWkJCUUVKYV9leGNHN18tVGlVTzFHT1R4VFJMZnIzM1BkMnhXLW13SHd4ZE04WFp0WUxnYVk0aWpGdGVxcElfdkFJMnFVSThQT1pMeHNpM2tUazIxT1lNUnlOQkNqalZjeTNueFhvaVNkRGtiNzR6QUZ1X0Z0VWc5S1prTUhFV0ZPV2lBZFhZd0o0aXQtWmNGRHpVNEVUczBXb1hoeVI3cE8xOU5yV3BZOG4?oc=5" target="_blank">India News Live Today October 21, 2024: \'Kashmir Pakistan nahi banega\': Farooq Abdullah\'s stern message after J&K attack</a>&nbsp;&nbsp;<font color="#6f6f6f">Hindustan Times</font></

In [1]:
import nltk

nltk.download("punkt")       # for tokenization
nltk.download("stopwords")   # for stopwords
nltk.download("wordnet")     # for lemmatization
nltk.download("omw-1.4")     # WordNet extra data


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True